# Homework III

Diogo Correia (ist199211) & Tomás Esteves (ist199341)

## I. Pen and Paper [12v]

**Consider the problem of learning a regression model from 5 univariate observations ((0.8), (1), (1.2), (1.4), (1.6)) with targets (24,20,10,13,12).**

In [ ]:
import pandas as pd

df1 = pd.read_csv("../data/hw3data.csv")

df1

### 1) [5v] Consider the basis function, $\phi_j(x) = x^j$ , for performing a 3-order polynomial regression,

$$
\hat{z} (x,w) = \sum_{j=0}^3 w_j \phi_j(x) = w_0 + w_1 x + w_2 x^2 + w_3 x^3
$$

Learn the Ridge regression ($l_2$ regularization) on the transformed data space using the closed form solution with $\lambda = 2$.

Hint: use numpy matrix operations (e.g., linalg.pinv for inverse) to validate your calculus

In [ ]:
import numpy.linalg as la
import numpy as np
from numpyarray_to_latex import to_ltx
from numpyarray_to_latex.jupyter import to_jup

In [ ]:
def ltx(array, **wargs):
    print(to_ltx(array, latexarraytype="bmatrix", **wargs))

In [ ]:
def jup(array, **wargs):
    to_jup(array, fmt="{:6.4f}")

In [ ]:
df_x = df1.drop("target", axis=1)
target = np.asmatrix(df1["target"]).transpose()

df_x["y0"] = df_x["y1"] ** 0
df_x["y2"] = df_x["y1"] ** 2
df_x["y3"] = df_x["y1"] ** 3

# swap columns
y1 = np.asarray(df_x[["y0", "y1", "y2", "y3"]])
ltx(y1)
to_jup(y1)

In [ ]:
y1_transposed = y1.transpose()
ltx(y1_transposed)
to_jup(y1_transposed)

In [ ]:
y1t_y1 = np.matmul(y1_transposed, y1)
ltx(y1t_y1)
to_jup(y1t_y1)

In [ ]:
y1t_y1_with_lambda = y1t_y1 + 2 * np.identity(4)
ltx(
    y1t_y1_with_lambda,
    mark_color="bblue",
    mark_elements=[
        (0, 0),
        (1, 1),
        (2, 2),
        (3, 3),
    ],
)
to_jup(y1t_y1_with_lambda)

In [ ]:
inverted = np.linalg.inv(y1t_y1_with_lambda)
ltx(inverted)
to_jup(inverted)

In [ ]:
inverted_mult = np.matmul(inverted, y1_transposed)
ltx(inverted_mult)
to_jup(inverted_mult)

In [ ]:
target

In [ ]:
w = np.matmul(inverted_mult, target)
ltx(w, mark_elements=[(i, 0) for i in range(4)])
to_jup(w)

In [ ]:
print("W0: ", w[0])
print("W1: ", w[1])
print("W2: ", w[2])
print("W3: ", w[3])

### 2) [1v] Compute the training RMSE for the learnt regression model.

In [ ]:
pred = np.matmul(y1, w)
ltx(pred)
to_jup(pred)

In [ ]:
to_jup(pred - target)

In [ ]:
distances = np.multiply((pred - target), (pred - target))
ltx(distances)
to_jup(distances)

In [ ]:
distances_sum = distances.sum()
distances_sum

In [ ]:
rmse = np.sqrt(distances_sum / 5)
rmse

### 3) [6v] Consider a multi-layer perceptron characterized by one hidden layer with 2 nodes. 

Using the activation function $f(x) = e^{0.1x}$ on all units, all weights initialized as 1 (including biases), and the half squared error loss.

Perform one batch gradient descent update (with learning rate $ \eta = 0.1$) for the first three observations (0.8), (1) and (1.2).

In [ ]:
df1_target = df1["target"]

df1

### Propagation

$$
W^{[1]} =  \begin{pmatrix}
1\\
1
\end{pmatrix}

W^{[2]} =  \begin{pmatrix}
1 & 1\\
\end{pmatrix}

b^{[1]} =  \begin{pmatrix}
1\\
1
\end{pmatrix}

b^{[2]} =  \begin{pmatrix}
1
\end{pmatrix}
$$

In [ ]:
learning_rate = 0.1

weight_1 = np.array([[1], [1]])
weight_2 = np.array([[1, 1]])

bias_1 = np.array([[1], [1]])
bias_2 = np.array([[1]])

y_values = np.array([df1["y1"][0:3]])
y_targets = np.array([df1["target"][0:3]])

to_jup(y_values)
to_jup(y_targets)

$$
z^{[1]} = W^{[1]} \cdot x + b^{[1]}\\

x^{[1]} = f(z^{[1]})
$$

In [ ]:
def z_1(x1):
    return np.dot(weight_1, x1) + bias_1

In [ ]:
z_1_values = z_1(y_values)

jup(z_1_values)

In [ ]:
def f(x):
    return np.e ** (0.1 * x)

In [ ]:
x_1_values = f(z_1_values)

jup(x_1_values)

$$
z^{[2]} = W^{[2]} \cdot x + b^{[2]}\\

x^{[2]} = f(z^{[2]})
$$

In [ ]:
def z_2(x2):
    return np.matmul(weight_2, x2) + bias_2

In [ ]:
z_2_values = z_2(x_1_values)

jup(z_2_values)

In [ ]:
x_2_values = f(z_2_values)

jup(x_2_values)

### Backpropagation

$$

E(x^{[2]},t) = \cfrac{1}{2}(x^{[2]} - t)^2

$$

$$
\cfrac{\delta E}{\delta x^{[2]}} = \cfrac{1}{2}(2x^{[2]} - 2t) = x^{[2]} - t\\

\cfrac{\delta x^{[i]}}{\delta z^{[i]}} = f'(z^{[i]}) = 0.1e^{0.1z^{[i]}}\\

\cfrac{\delta z^{[i]}}{\delta W^{[i]}} = x^{[i-1]}\\

\cfrac{\delta z^{[i]}}{\delta b^{[i]}} = 1\\

\cfrac{\delta z^{[i]}}{\delta x^{[i-1]}} = W^{[i]}\\

\cfrac{\delta E}{\delta W^{[i]}} = \delta^{[i]} \cfrac{\delta z^{[i]}}{\delta W^{[i]}} = \delta^{[i]} (x^{[i-1]})^T\\

$$

$$

\text{Output layer:}\\

\delta^{[2]} = \cfrac{\delta E}{\delta x^{[2]}} \cfrac{\delta x^{[2]}}{\delta z^{[2]}}\\

\text {Other layers:}\\

\delta^{[i]} = (\cfrac{\delta z^{[i+1]}}{\delta x^{[i]}})^T \delta^{[i+1]} \cfrac{\delta x^{[i]}}{\delta z^{[i]}} 


$$

$$

\delta^{[2]} = (x^{[2]} - t) \circ 0.1e^{0.1z^{[2]}}\\

\delta^{[1]} = (W^{[2]})^T \delta^{[2]} \circ 0.1e^{0.1z^{[1]}}\\

$$

In [ ]:
def f_derivative(x):
    return 0.1 * np.e ** (0.1 * x)

In [ ]:
def delta_2_fn(x_2, t, z_2):
    return (x_2 - t) * 0.1 * np.e ** (0.1 * z_2)

In [ ]:
def delta_n_fn(w_n_plus_1, delta_n_plus_1, z_n):
    return np.multiply((w_n_plus_1.transpose() * delta_n_plus_1), f_derivative(z_n))

In [ ]:
delta_2_values = delta_2_fn(x_2_values, y_targets, z_2_values)

jup(delta_2_values)

In [ ]:
delta_1_values = delta_n_fn(weight_2, delta_2_values, z_1_values)

jup(delta_1_values)

$$
W_{new}^{[i]} = W_{old}^{[i]} - \eta \cfrac{\delta E} {\delta w^{[i]}} = W_{old}^{[i]}- \eta \delta^{[i]} (x^{[i-1]})^T\\
$$

In [ ]:
def weight_fn(w_old_n, eta, delta_n, x_n_minus_1):
    return w_old_n - eta * np.matmul(delta_n, x_n_minus_1.transpose())

In [ ]:
def print_delta_weights(learning_rate, delta_n, x_n_minus_1):
    for i in range(delta_n.shape[1]):
        jup(
            -learning_rate * np.matmul(delta_n[:, [i]], x_n_minus_1[:, [i]].transpose())
        )

In [ ]:
print_delta_weights(learning_rate, delta_2_values, x_1_values)

In [ ]:
new_weight_2 = weight_fn(weight_2, learning_rate, delta_2_values, x_1_values)

jup(new_weight_2)

In [ ]:
print_delta_weights(learning_rate, delta_1_values, y_values)

In [ ]:
new_weight_1 = weight_fn(weight_1, learning_rate, delta_1_values, y_values)

jup(new_weight_1)

$$
b_{new}^{[i]} = b_{old}^{[i]} - \eta \cfrac{\delta E} {\delta b^{[i]}}
$$

In [ ]:
def bias_fn(b_old_n, eta, delta_n):
    return b_old_n - eta * np.matmul(
        delta_n, np.vstack(np.repeat([1], delta_1_values.shape[1]))
    )

In [ ]:
def print_delta_biases(learning_rate, delta_n):
    for i in range(delta_n.shape[1]):
        jup(-learning_rate * delta_n[:, [i]])

In [ ]:
print_delta_biases(learning_rate, delta_2_values)

In [ ]:
new_bias_2 = bias_fn(bias_2, learning_rate, delta_2_values)

jup(new_bias_2)

In [ ]:
print_delta_biases(learning_rate, delta_1_values)

In [ ]:
new_bias_1 = bias_fn(bias_1, learning_rate, delta_1_values)

jup(new_bias_1)

Propagation
$$
W^{[1]} =  \begin{pmatrix}
1.08062041\\
1.08062041
\end{pmatrix}

W^{[2]} =  \begin{pmatrix}
1.8518485 & 1.8518485\\
\end{pmatrix}

b^{[1]} =  \begin{pmatrix}
1.08518485\\
1.08518485
\end{pmatrix}

b^{[2]} =  \begin{pmatrix}
1.7012589
\end{pmatrix}

$$

## Programming and Critical Anlaysis[8v]

**Consider the following three regressors applied on kin8nm.arff data (available at the webpage):**

- linear regression with Ridge regularization term of 0.1
- two MLPs
     - 𝑀𝐿𝑃1 and 𝑀𝐿𝑃2 
- each with two hidden layers of size 10, hyperbolic tangent function as the activation function of all nodes, a maximum of 500 iterations, and a fixed seed (random_state=0). 
- 𝑀𝐿𝑃1 should be parameterized with early stopping while 𝑀𝐿𝑃2 should not consider early stopping. 

Remaining parameters (e.g., loss function, batch size, regularization term, solver) should be set as default

Using a 70-30 training-test split with a fixed seed (random_state=0):

### 4) [4v] **Compute the MAE of the three regressors: linear regression, 𝑀𝐿𝑃1 and 𝑀𝐿𝑃2.**

In [ ]:
from scipy.io.arff import loadarff
import pandas as pd
from sklearn import model_selection, metrics
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Reading the ARFF file
data = loadarff("../data/kin8nm.arff")
df = pd.DataFrame(data[0])

df.head()

In [ ]:
# Separate features from the outcome (class)
X = df.drop("y", axis=1)
y = df["y"]

y.head()

In [ ]:
# Split the dataset into a training set (70%) and a testing set (30%)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X.values, y.values, train_size=0.7, random_state=0
)

In [ ]:
rr = Ridge(alpha=0.01)

In [ ]:
mlp1 = MLPRegressor(
    hidden_layer_sizes=(10, 10),
    activation="tanh",
    max_iter=500,
    random_state=0,
    early_stopping=True,
)

In [ ]:
mlp2 = MLPRegressor(
    hidden_layer_sizes=(10, 10),
    activation="tanh",
    max_iter=500,
    random_state=0,
    early_stopping=False,
)

In [ ]:
rr.fit(X_train, y_train)
mlp1.fit(X_train, y_train)
mlp2.fit(X_train, y_train)

In [ ]:
rr_pred = rr.predict(X_test)
mlp1_pred = mlp1.predict(X_test)
mlp2_pred = mlp2.predict(X_test)

In [ ]:
print("Ridge Regularization MAE:", metrics.mean_absolute_error(y_test, rr_pred))

In [ ]:
print("MLP1 Regularization MAE:", metrics.mean_absolute_error(y_test, mlp1_pred))

In [ ]:
print("MLP2 Regularization MAE:", metrics.mean_absolute_error(y_test, mlp2_pred))

### 5) [1.5v] **Plot the residues (in absolute value) using two visualizations: boxplots and histograms.**

Hint: consider using boxplot and hist functions from matplotlib.pyplot to this end

In [ ]:
rr_residues = []
mlp1_residues = []
mlp2_residues = []

for i in range(0, len(y_test)):
    rr_residues.append(abs(y_test[i] - rr_pred[i]))
    mlp1_residues.append(abs(y_test[i] - mlp1_pred[i]))
    mlp2_residues.append(abs(y_test[i] - mlp2_pred[i]))

In [ ]:
df = pd.DataFrame({"Ridge": rr_residues, "MLP1": mlp1_residues, "MLP2": mlp2_residues})

df.head()

In [ ]:
sns.boxplot(data=df)

plt.show()

In [ ]:
sns.histplot(data=df)

plt.show()

### 6) [1v] **How many iterations were required for 𝑀𝐿𝑃1 and 𝑀𝐿𝑃2 to converge?**

In [ ]:
print("MLP1 number of iterations:", mlp1.n_iter_)

In [ ]:
print("MLP2 number of iterations:", mlp2.n_iter_)

### 7) [1.5v] **What can be motivating the unexpected differences on the number of iterations?**

**Hypothesize one reason underlying the observed performance differences between the MLPs.**

Read more about the MLP regressor at https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html

By comparing the number of iteration of both models, we can conclude that MLP1 takes a lot more iterations (452 > 77) to converge than MLP2. This is because MLP1 uses early stopping, which means that the model stops training when the validation score is not improving anymore. By reading the documentation about the [sklearn.neural_network.MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html), we can see that the default value for the validation fraction is `validation_fraction` (default = 0.1), which means that 10% of the training data is used for validation and the other 90% data to train the model. After each iteration, the model will calculate the validation score. If the score is not improving by at least `tol` (default value = 1e-4) for `n_iter_no_change` (default value = 10), the model will stop training. This will allow the MLP1 to have better performance (lower MAE value, and lower residue values) than MLP2, but it will take a lot more iterations to converge.

Since the MLP1 uses early stopping and MLP2 does not, we can attribute the difference in performance to the early stopping. 
Due to how the early stopping works, similar to splitting the data into training and testing, early stopping splits the training data into training and validation, this will train a model that will learn from the training data and will evaluate it with the valuation data. This will cause the model to learn more effectively, which will cause the model to be more robust and generalize better. This is why the MLP1 has a lower MAE value and lower residue values than MLP2.

In [ ]:
## this takes 1m32sec to run on my machine, so don't run it unless you have time to spare :P

validation_scores = []
for n_iter in range(1, 500, 10):
    mlp = MLPRegressor(
        hidden_layer_sizes=(10, 10),
        activation="tanh",
        max_iter=n_iter,
        random_state=0,
        early_stopping=True,
    )
    mlp.fit(X_train, y_train)
    validation_scores.append(mlp.score(X_train, y_train))

validation_scores